# Data Journalism Lesson 16: Waffle charts

Making quick graphics for reporting.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('pywaffle')
await micropip.install('matplotlib')
await micropip.install("nbformat>=4.2.0")

In [ ]:
from IPython.display import display, HTML
import pandas as pd

# --- Simple Grading/Checking Functions ---
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_dict_equality(inputted, expected, exercise_name):
    if inputted == expected:
        display_feedback(True, f'Great job! The dictionary for {exercise_name} is correct.', '')
    else:
        display_feedback(False, f'The dictionary for {exercise_name} is not correct. Expected {expected}, got {inputted}', 'Please check your data and try again.')

def check_value(inputted, expected, value_name):
    if inputted == expected:
        display_feedback(True, f'Correct! The value for {value_name} is {inputted}.', '')
    else:
        display_feedback(False, f'The value for {value_name} is not correct. Expected {expected}, got {inputted}.', 'Please check and try again.')

def check_df_shape_and_columns(df, expected_shape, expected_columns, df_name):
    shape_correct = df.shape == expected_shape
    cols_correct = sorted(list(df.columns)) == sorted(expected_columns)
    if shape_correct and cols_correct:
        display_feedback(True, f'{df_name} DataFrame has the correct shape and columns.', '')
    else:
        messages = []
        if not shape_correct:
            messages.append(f'Expected shape {expected_shape}, got {df.shape}.')
        if not cols_correct:
            messages.append(f'Expected columns {expected_columns}, got {sorted(list(df.columns))}.')
        display_feedback(False, ' '.join(messages), f'Please check your {df_name} DataFrame.')

def check_plot_params(params, expected_params, plot_name):
    correct = True
    feedback_messages = []
    for p_name, p_val_expected in expected_params.items():
        p_val_actual = params.get(p_name)
        if p_name == 'values': # Special handling for dict comparison
            if isinstance(p_val_actual, pd.Series):
                 p_val_actual = p_val_actual.to_dict()
            if p_val_actual == p_val_expected:
                feedback_messages.append(f'Correct {p_name} for {plot_name}.')
            else:
                correct = False
                feedback_messages.append(f'Incorrect {p_name} for {plot_name}. Expected {p_val_expected}, got {p_val_actual}.')
        elif p_val_actual == p_val_expected:
            feedback_messages.append(f'Correct {p_name} for {plot_name}.')
        else:
            correct = False
            feedback_messages.append(f'Incorrect {p_name} for {plot_name}. Expected \'{p_val_expected}\', got \'{p_val_actual}\'.')
    
    final_message_correct = f'Plot parameters for {plot_name} are correct!'
    final_message_incorrect = ' '.join(feedback_messages)
    display_feedback(correct, final_message_correct, final_message_incorrect)

In [ ]:
# --- State Setup and Data Loading ---
default_state_abbr = 'MN'
state_full_name = "Minnesota"


legislatures_df = pd.read_csv("../_static/state-legislatures/legislatures.csv")

# U.S. House data
house_data_expected = {"Republicans": 220, "Democrats": 212, "Vacant": 3}

# State legislature data preparation
xgr_df = legislatures_df[legislatures_df['state'] == state_full_name].copy()
xgr_rows_expected = len(xgr_df)

senate_df = xgr_df[['state', 'senate_dem', 'senate_rep', 'senate_other', 'senate_vacant']].copy()
senate_df = senate_df.melt(id_vars=['state'], 
                            value_vars=['senate_dem', 'senate_rep', 'senate_other', 'senate_vacant'],
                            var_name='party',
                            value_name='members')
# Filter out rows where members might be NaN or 0 if not relevant for waffle (pywaffle typically filters 0s)
senate_df = senate_df[senate_df['members'].notna() & (senate_df['members'] > 0)].reset_index(drop=True)
senate_df_expected_cols = ['state', 'party', 'members']

# Most partisan data preparation
most_partisan_df = legislatures_df.copy()
most_partisan_df['partisan_lean'] = most_partisan_df.apply(
    lambda row: row['senate_dem'] / row['total_senate'] if row['senate_dem'] > row['senate_rep'] 
    else (row['senate_rep'] / row['total_senate'] if row['total_senate'] > 0 else 0), axis=1
)
most_partisan_top10_df = most_partisan_df.nlargest(10, 'partisan_lean')
most_partisan_senate_df = most_partisan_top10_df[['state', 'senate_dem', 'senate_rep', 'senate_other', 'senate_vacant']].copy()
most_partisan_senate_df = most_partisan_senate_df.melt(id_vars=['state'],
                                                        value_vars=['senate_dem', 'senate_rep', 'senate_other', 'senate_vacant'],
                                                        var_name='party',
                                                        value_name='members')
most_partisan_senate_df = most_partisan_senate_df[most_partisan_senate_df['members'].notna() & (most_partisan_senate_df['members'] > 0)].reset_index(drop=True)

# Combine for xgrs_df
if state_full_name in most_partisan_senate_df['state'].unique():
    xgrs_df = most_partisan_senate_df.copy()
else:
    xgrs_df = pd.concat([most_partisan_senate_df, senate_df[senate_df['state'] == state_full_name]], ignore_index=True)

xgrs_df_expected_cols = ['state', 'party', 'members']

In [ ]:
from myst_nb import glue

glue("state_full_name", state_full_name, display=False)
glue("xgr_rows", xgr_rows_expected, display=False)
glue("house_data_expected_str", str(house_data_expected), display=False) # For checking exercises

## The Goal

In this lesson, you'll learn how to create waffle charts, an effective alternative to pie charts for visualizing proportions. By the end of this tutorial, you'll understand how to use the `pywaffle` library in Python to construct single and comparative waffle charts. You'll practice creating these charts with real data, learning how to adjust chart parameters for clarity and how to handle uneven data comparisons. This skill will enable you to present proportional data in a clear, visually appealing format that's easier to interpret than traditional pie charts.

## Why Visualize Data?

Show me someone who has a job in data visualization, and I'll show you someone who has Edward Tufte's *The Visualization of Quantitative Information* on their bookshelf. It's one of the most popular, and most influential books on data visualization ever written. As such, it's one of the most referenced, and most criticized, books on the subject. If visualizing data is interesting to you, you have to read it.

What will you find? A clearly articulated argument of what makes a data graphic good. So clearly articulated that some call it stifling, rigid, inflexible. Some of his commandments can be taken to extremes, where instead of intelligent and clear graphical excellence, as he describes it, you get something so esoteric it doesn't make sense.

But once you realize Tufte -- in spite of his unambiguous language -- is more like the pirate's code in Pirate's of the Caribbean (more of a guideline than a rule), then you'll see the genius of it all.

Indeed, page 1, part 1, gives you a taste of his direct style.

"EXCELLENCE in statistical graphics consists of complex ideas communicated with clarity, precision, and efficiency. Graphical displays should

-   show the data
-   induce the viewer to think about the substance rather than about methodology, graphic design, the technology of graphic production, or something else
-   avoid distorting what the data have to say
-   present many numbers in a small space
-   make large data sets coherent
-   encourage the eye to compare different pieces of data
-   reveal the data at several levels of detail, from a broad overview to the fine structure
-   serve a reasonably clear purpose: description, exploration, tabulation, or decoration
-   be closely integrated with the statistical and verbal descriptions of a data set. "

If you can do all that, you're on a great path. Easy right? Not exactly. Doing some of this is easy with the tools we have. Some of this requires a lot of thought.

That brings us to today when we come not to praise the pie chart but to bury it. They're terrible, and people abuse them horribly. Using Tufte's own words, pie charts regularly fail to show the data, often distorting it, and are truly horrible at making large datasets coherent.

This can lead to what Tufte calls failures of Graphical Integrity. Tufte insists that the early years of graphics were plagued with designers who used them to lie -- intentionally or not -- by covering up bad data or bad results with charts. He credits no less than John Tukey for making graphics respectable again in the late 1960s.

I'm here to argue that pie charts are the tool of the liar. But showing proportion of a whole is a very important job for a graphic. Fortunately, there are better tools. That tool is called the waffle chart, and we introduce those here.

As you're working today, keep Tufte's six principles of graphical integrity in mind:

"Graphical integrity is more likely to result if these six principles are followed:

-   The representation of numbers, as physically measured on the surface of the graphic itself, should be directly proportional to the numerical quantities represented.
-   Clear, detailed, and thorough labeling should be used to defeat graphical distortion and ambiguity. Write out explanations of the data on the graphic itself. Label important events in the data.
-   Show data variation, not design variation.
-   In time-series displays of money, deflated and standardized units of monetary measurement are nearly always better than nominal units.
-   The number of information-carrying (variable) dimensions depicted should not exceed the number of dimensions in the data.
-   Graphics must not quote data out of context."

## The Basics

Pie charts are the devil. They should be an instant F in any data visualization class. The problem with them? How carefully can you evaluate angles and area? Unless they are blindingly obvious and only a few categories, not well. If you've got 25 categories, how can you tell the difference between 7 and 9 percent? You can't.

So let's introduce a better way: The waffle chart. Why a waffle chart? Because it's made up of small squares, just like a waffle. Some call it a square pie chart. I personally hate that. Waffles it is.

A waffle chart is similar to a stacked bar chart. **A waffle chart is designed to show you parts of the whole -- proportionality**. How many things make up the whole. It will also show magnitude, just like a stacked bar.

What's the difference?

The truth is, there really isn't one. If I had to stand before a judge and come up with an answer or go to jail, I'd say that a stacked bar chart can get more things in it -- more counties, more states, more schools, etc. The waffle chart demands focus -- which you'll see in a minute -- and so you can really only handle a small number of waffle charts at once. Two is great. Three is fine. Five is pushing it. Ten has to be something really special. But with a stacked bar, 10 is easily doable.

What that boils down to is that a waffle chart is a visually different form that accomplishes the same goal as a stacked bar chart, but works best when there's only a couple of things to compare. A two-bar stacked bar chart would look silly. A two-stack waffle chart would look great.

Waffle charts are not built into common high-level plotting libraries like Plotly Express in the same way bar charts are. Instead, we use specialized libraries like `pywaffle`. `pywaffle` is built on top of Matplotlib, Python's foundational plotting library. While Matplotlib is very powerful, `pywaffle` makes creating waffle charts much simpler.

The `pywaffle` library might have a slightly different approach than libraries you've used before. It often involves creating a dictionary of your data and then passing it to the `Waffle` class.

To start, we load the necessary libraries: `pandas` for data handling, `matplotlib.pyplot` for basic plotting framework, and `Waffle` from `pywaffle`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pywaffle import Waffle

We're going to try this waffle chart several different ways. The first way is by directly creating the data as a Python dictionary.

For this, we're going to look at the partisan breakdown of the U.S. House of Representatives. The House, without editorializing, is a mess, precisely because it's almost evenly divided (history nerd note: The House is supposed to be a mess. It was designed that way by the Founders. This is nothing new). For any measure to pass the House, the majority party has to have all of its members on board. They can't have people voting against. So what does that look visually?

First, we'll make our dictionary. The pattern is `{"Category Name": value}`. So there's 220 Republican members of the House, 212 Democrats and 3 seats are vacant as of this writing.

In [ ]:
house_data = ____ # Fill in the dictionary: e.g., {"Category1": value1, "Category2": value2, ...}

display(house_data)
check_dict_equality(house_data, house_data_expected, 'U.S. House Data')

So what does the breakdown look like visually?

### Exercise 1: Make a waffle chart

The `pywaffle` library can break this down in a way that's easier on the eyes than a pie chart. We use `plt.figure()` and pass `FigureClass=Waffle`. Key parameters for `Waffle` include:
- `values`: The dictionary or list of data values.
- `rows`: The number of rows in the waffle grid (10 is a good start).
- `title`: For the chart title.
- `labels`: For the legend.
- `colors`: A list or tuple of colors. Here, red for Republicans, blue for Democrats, and grey for vacant.
- `legend`: A dictionary to control legend properties.

We'll also use `plt.xlabel()` to add a label.

In [ ]:
fig_ex1 = plt.figure(
    FigureClass=Waffle,
    values=____, 
    rows=____, 
    colors=____, # Tuple of colors: e.g., ("red", "blue", "grey")
    title={'label': 'Partisan breakdown of the House', 'loc': 'left', 'fontsize': 14},
    labels=[f"{k} ({v})" for k, v in house_data.items()],
    legend={'loc': 'lower left', 'bbox_to_anchor': (0, -0.25), 'ncol': len(house_data), 'framealpha': 0.7, 'fontsize': 10}
)
plt.xlabel("1 square = 1 representative", fontsize=10)
plt.show()

Now, this chart is attempting to put 435 dots on a chart, so they get kind of small. Can we control that? The answer is: a little. How do we do that? Division.

### Exercise 2: Divide by

`pywaffle` automatically scales the blocks to fit the figure. If we want each square to represent more than one unit, we need to divide our data values *before* passing them to `Waffle`. Want one dot to equal 3 members of Congress? Divide your values by 3. Note, however -- there are three vacancies. If you divide by 3, `3 // 3` is 1, so it will appear as one block. If we divided by 10, `3 // 10` (using integer division) would be 0, and it wouldn't appear. Let's solve that by dividing by 3.

You'll need to create a new dictionary `house_data_divided` where each value is divided by 3 (use integer division `//`).

In [ ]:
# house_data = {"Republicans": 220, "Democrats": 212, "Vacant": 3}
division_factor = 3
house_data_divided = {key: value ____ division_factor for key, value in house_data.items()}

fig_ex2 = plt.figure(
    FigureClass=Waffle,
    values=____, # Use the divided data
    rows=10, 
    colors=("red", "blue", "grey"), 
    title={'label': 'Partisan breakdown of the House (scaled)', 'loc': 'left', 'fontsize': 14},
    labels=[f"{k} (~{v*division_factor})" for k, v in house_data_divided.items()], # Adjust labels to reflect original numbers
    legend={'loc': 'lower left', 'bbox_to_anchor': (0, -0.25), 'ncol': len(house_data_divided), 'framealpha': 0.7, 'fontsize': 10}
)
plt.xlabel(f"1 square = {division_factor} representatives", fontsize=10)
plt.show()

And now you have a waffle chart that shows the partisan breakdown of the People's House. A very good question to ask yourself right now -- is it a *good* chart? What story is it telling? Is that clear? Here's another question -- do we need this chart to tell this story? The House is nearly evenly divided. We've made a chart, dividing by 3, where it would take a beat or two for you to see that there's almost exactly the same number of dots for each party. Do we need to represent that visually?

I would say no. If your chart doesn't show some kind of difference -- tell some kind of story -- then a sentence in your story will do the same job with less code and with the same impact.

## Now With DataFrames and `pywaffle`

What does it look like if we try to use a pandas DataFrame instead of a manually created dictionary? And how can we create multiple waffle charts, similar to faceting in other libraries?

First, for this to work with `pywaffle`, your data typically needs to be in a format where you can easily extract categories (for labels/colors) and corresponding values. A dictionary is often the most direct input, but you can create this dictionary from a DataFrame.

Instead of Congress, let's look at the partisan breakdown of the {glue:text}`state_full_name` Senate. Why the senate and not the house? Well, not every state has a house (cough, cough Nebraska), so to make it easier, we'll focus on the chamber everyone has. This data comes from the National Conference of State Legislature's data on partisan control of state legislatures.

We're going to grab the data and filter all in one step. You just need to ensure your `legislatures_df` is loaded and then filter it for the state: {glue:text}`state_full_name`.

In [ ]:
legislatures_df = pd.read_csv("../_static/state-legislatures/legislatures.csv")
current_state_legislature_df = legislatures_df[legislatures_df['state'] == ____].copy()

display(current_state_legislature_df)
check_value(len(current_state_legislature_df), xgr_rows_expected, f'{state_full_name} legislature data row count')

Let's take a quick look at it. It won't take long -- it's usually one row of data per state. FYI: The data name `xgr` used in the example is one of those old-timey journalism terms from way back in the day. In the Times of Yore, legislature stories from the AP all started with the slug XGR.

In [ ]:
display(____.head())

The problem we have is that it's one row (wide format) -- and it's got lots of things we don't want or need for the waffle chart values. First, we want to select only senate-related columns. Then, we'll use pandas' `melt` function to make our wide data long. With `melt`, you specify `id_vars` (columns to keep as identifiers, like 'state') and `value_vars` (columns to unpivot). We also give names to the new columns for party and member count.

We will store this in `state_senate_long_df`.

In [ ]:
columns_to_select = ['state', 'senate_dem', 'senate_rep', 'senate_other', 'senate_vacant']
selected_state_data = current_state_legislature_df[columns_to_select].copy()

state_senate_long_df = selected_state_data.melt(
    id_vars=____, # Column to keep (e.g. 'state')
    value_vars=____, # Columns to unpivot (e.g. ['senate_dem', ...])
    var_name='party',
    value_name='members'
)
# Remove rows where 'members' is 0 or NaN as they don't contribute to the waffle chart
state_senate_long_df = state_senate_long_df[state_senate_long_df['members'].notna() & (state_senate_long_df['members'] > 0)].reset_index(drop=True)

display(state_senate_long_df.head())
check_df_shape_and_columns(state_senate_long_df, senate_df[senate_df['state']==state_full_name].shape, senate_df_expected_cols, f'{state_full_name} Senate Long DF')

Now take a look at `state_senate_long_df`:

In [ ]:
display(____.head())

Now we have data in a long format. We can convert this into a dictionary suitable for `pywaffle`.

### Exercise 3: Waffle chart from DataFrame

Create a waffle chart for your state's senate. You'll first need to create a dictionary from the `state_senate_long_df` where keys are 'party' and values are 'members'. Then use this dictionary with `pywaffle`.

We'll define a color map for consistency.

In [ ]:
party_color_map = {
    'senate_dem': 'blue',
    'senate_rep': 'red',
    'senate_other': 'orange', # Or 'green', 'purple' etc.
    'senate_vacant': 'grey'
}

# Create dictionary from the DataFrame
state_senate_data_dict = pd.Series(____.values, index=____).to_dict()

# Prepare colors in the order of the dictionary items
plot_colors_ex3 = [party_color_map.get(party, 'black') for party in state_senate_data_dict.keys()]

fig_ex3 = plt.figure(
    FigureClass=Waffle,
    values=state_senate_data_dict,
    rows=5, # n_rows equivalent
    colors=plot_colors_ex3,
    title={'label': f'{state_full_name} Senate Partisan Breakdown', 'loc': 'left', 'fontsize': 12},
    labels=[f"{k.replace('senate_', '').capitalize()} ({v})" for k, v in state_senate_data_dict.items()],
    legend={'loc': 'lower left', 'bbox_to_anchor': (0, -0.3), 'ncol': len(state_senate_data_dict), 'framealpha': 0.7},
    block_arranging_style='snake', # Example: try 'normal', 'new-line'
    figsize=(7, 4)
)
plt.show()

Now we have a similar chart for your state's senate. Which party has the majority? Is that clear from your chart?

## The Recap

Throughout this lesson, you've learned how to create and customize waffle charts using Python's `pywaffle` library with Matplotlib. You've practiced making single waffle charts to display proportions within a dataset, such as the partisan makeup of the U.S. House of Representatives. You've also learned how to create comparative waffle charts by manually setting up subplots, allowing you to visually compare proportions across different facets (states). You've discovered how to adjust your data and chart parameters to handle issues like scale differences and visual clarity. Remember, waffle charts offer a more precise and visually appealing alternative to pie charts for displaying proportional data, making them a valuable tool in your data visualization toolkit.

## Terms to Know

-   **Waffle chart**: A chart type that represents data as squares (or other icons) in a grid, where each square represents a proportion of the whole.
-   **`pywaffle`**: A Python library for creating waffle charts, built on Matplotlib.
-   **`matplotlib.pyplot`**: A collection of functions that make Matplotlib work like MATLAB, providing a convenient way to create plots and visualizations.
-   **`plt.figure(FigureClass=Waffle, ...)`**: The primary way to create a waffle chart using `pywaffle`, by specifying `Waffle` as the figure class.
-   **`values` parameter (in `Waffle`)**: A dictionary, list, or pandas Series containing the data values for the waffle chart segments.
-   **`rows` parameter (in `Waffle`)**: Determines the number of rows in the waffle grid.
-   **`colors` parameter (in `Waffle`)**: A list or tuple specifying the colors for each category in the data.
-   **`labels` parameter (in `Waffle`)**: Used to provide labels for the legend.
-   **`legend` parameter (in `Waffle`)**: A dictionary to customize the legend's appearance and position.
-   **Python Dictionary**: A data structure used to store data as key-value pairs, often used as input for `pywaffle`.
-   **Pandas DataFrame `melt()`**: A function to transform a DataFrame from wide format to long format, useful for preparing data for some types of plotting.
-   **Matplotlib Subplots (`plt.subplots()`)**: Used to create a figure with multiple individual plots (axes) arranged in a grid, allowing for manual faceting.